In [1]:
import numpy as np
import pandas as pd
import pingouin as pg
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

# Round 1

#### Maria round 1

In [2]:

df = pd.read_csv("Inputs/round_1_maria_segmental.csv", header=0)

print(df['bowelSegmentName'].unique())

# ILEUM AND JEJUNUM ARE STRATIFIED INTO 10 SEGMENTS (terminal ileum is separate)
# Merge ileum and jejunum subsegments, picking the highest score
df['patient_segment_scorer'] = df['patientIdentifier'] + ' ' + df['newBowelSegmentName'] + ' ' + df['scorerEmail']
df = df.sort_values('segmentalScore', ascending=False)
df = df.drop_duplicates(subset=['patient_segment_scorer'], keep='first')
print('\nSCORES count:', df.value_counts('patient_segment_scorer').value_counts())
print('\nNEW bowel segment names:', df['newBowelSegmentName'].unique())

# make target to be patientIdentifier + scorerEmail
df['patient_segment'] = df['patientIdentifier'] + ' ' + df['newBowelSegmentName']
df = df[['patient_segment', 'scorerEmail', 'segmentalScore']]

# ALL SEGMENTS
# ICC for all segmental scores combined, with patient_segment as target
all_segments_icc_r1_maria = pg.intraclass_corr(data=df, targets='patient_segment', raters='scorerEmail',
                            ratings='segmentalScore', nan_policy='omit')
all_segments_icc_r1_maria = all_segments_icc_r1_maria.loc[2,:]
print('\nICC among all segmental scores combined - Round 1 Maria:')
print(all_segments_icc_r1_maria[['ICC', 'CI95%']])

# PER SEGMENT
segments = ['Stomach', 'Duodenum', 'Jejunum', 'Ileum', 'Terminal ileum', 'Right colon', 'Transverse colon', 'Descending colon', 'Sigmoid colon', 'Rectum']

# initialize an empty DataFrame to store results
per_segment_r1_maria = pd.DataFrame(columns=['ICC', 'CI95%'], index=segments)

for segment in segments:
    # select only rows where patient_segment contains the "segment" string
    df_segment = df[df['patient_segment'].str.contains(segment)]
    if len(df_segment['segmentalScore'].unique()) < 2:
        per_segment_r1_maria.at[segment, 'ICC'] = np.nan
        per_segment_r1_maria.at[segment, 'CI95%'] = f"[all scores = {df_segment['segmentalScore'].unique()[0]}]"
        continue
    icc = pg.intraclass_corr(data=df_segment, targets='patient_segment', raters='scorerEmail', ratings='segmentalScore', nan_policy='omit')
    icc = icc.loc[2,:]
    # append the result to the DataFrame
    per_segment_r1_maria.at[segment, 'ICC'] = icc['ICC']
    per_segment_r1_maria.at[segment, 'CI95%'] = icc['CI95%']



['Stomach' 'Duodenum' 'Jejunum 1' 'Jejunum 2' 'Jejunum 3' 'Jejunum 4'
 'Jejunum 5' 'Jejunum 6' 'Jejunum 7' 'Jejunum 8' 'Jejunum 9' 'Jejunum 10'
 'Ileum 1' 'Ileum 2' 'Ileum 3' 'Ileum 4' 'Ileum 5' 'Ileum 6' 'Ileum 7'
 'Ileum 8' 'Ileum 9' 'Terminal ileum' 'Caecum' 'Ascending colon'
 'Transverse colon' 'Descending colon' 'Sigmoid colon' 'Rectum'
 'Pre-terminal ileum']

SCORES count: count
1    400
Name: count, dtype: int64

NEW bowel segment names: ['Ileum' 'Terminal ileum' 'Right colon' 'Sigmoid colon' 'Jejunum' 'Rectum'
 'Stomach' 'Duodenum' 'Descending colon' 'Transverse colon']

ICC among all segmental scores combined - Round 1 Maria:
ICC          0.555894
CI95%    [0.46, 0.66]
Name: 2, dtype: object


#### Lemann round 1

In [3]:
# Segmental LEMANN round 1

df = pd.read_csv("Inputs/round_1_lemann_segmental.csv", header=0)

print(df['bowelSegmentName'].unique())

# ILEUM AND JEJUNUM ARE STRATIFIED INTO 10 SEGMENTS (terminal ileum = ILEUM 10)
# Change bowel segment names to be consistent with MARIA
df['bowelSegmentName'] = df['bowelSegmentName'].replace({'Ileum 10': 'Terminal ileum'})
df['bowelSegmentName'] = df['bowelSegmentName'].str.replace(r'\d+', '', regex=True)
df['bowelSegmentName'] = df['bowelSegmentName'].str.strip()

# Merge ileum and jejunum subsegments, picking the highest score
df['patient_segment_scorer'] = df['patientIdentifier'] + ' ' + df['bowelSegmentName'] + ' ' + df['scorerEmail']
df = df.sort_values('segmentIndex', ascending=False)
df = df.drop_duplicates(subset=['patient_segment_scorer'], keep='first')
print('\nSCORES count:', df.value_counts('patient_segment_scorer').value_counts())
print('\nNEW bowel segment names:', df['bowelSegmentName'].unique())

# Make target to be patientIdentifier + scorerEmail
df['patient_segment'] = df['patientIdentifier'] + ' ' + df['bowelSegmentName']
df = df[['patient_segment', 'scorerEmail', 'segmentIndex']]

df['segmentIndex'] = df['segmentIndex'].astype(float)

# ALL SEGMENTS
all_segments_icc_r1_lemann = pg.intraclass_corr(data=df, targets='patient_segment', raters='scorerEmail',
                            ratings='segmentIndex', nan_policy='omit')
all_segments_icc_r1_lemann = all_segments_icc_r1_lemann.loc[2,:]
print('\nAll segments combined - Round 1 Lemann:')
print(all_segments_icc_r1_lemann[['ICC', 'CI95%']])

# PER SEGMENT
segments = ['Stomach', 'Duodenum', 'Jejunum', 'Ileum', 'Terminal ileum', 'Caecum', 'Ascending colon', 'Transverse colon', 'Descending colon', 'Sigmoid colon', 'Rectum']

per_segment_r1_lemann = pd.DataFrame(columns=['ICC', 'CI95%'], index=segments)

for segment in segments:
    # select only rows where patient_segment contains the "segment" string
    df_segment = df[df['patient_segment'].str.contains(segment)]
    if len(df_segment['segmentIndex'].unique()) < 2:
        per_segment_r1_lemann.at[segment, 'ICC'] = np.nan
        per_segment_r1_lemann.at[segment, 'CI95%'] = f"[all scores = {df_segment['segmentIndex'].unique()[0]}]"
        continue
    icc = pg.intraclass_corr(data=df_segment, targets='patient_segment', raters='scorerEmail', ratings='segmentIndex', nan_policy='omit')
    icc = icc.loc[2,:]
    per_segment_r1_lemann.at[segment, 'ICC'] = icc['ICC']
    per_segment_r1_lemann.at[segment, 'CI95%'] = icc['CI95%']


['Ascending colon' 'Caecum' 'Descending colon' 'Duodenum' 'Ileum 1'
 'Ileum 10' 'Ileum 2' 'Ileum 3' 'Ileum 4' 'Ileum 5' 'Ileum 6' 'Ileum 7'
 'Ileum 8' 'Ileum 9' 'Jejunum 1' 'Jejunum 10' 'Jejunum 2' 'Jejunum 3'
 'Jejunum 4' 'Jejunum 5' 'Jejunum 6' 'Jejunum 7' 'Jejunum 8' 'Jejunum 9'
 'Rectum' 'Sigmoid colon' 'Stomach' 'Transverse colon']

SCORES count: count
1    440
Name: count, dtype: int64

NEW bowel segment names: ['Terminal ileum' 'Ileum' 'Jejunum' 'Sigmoid colon' 'Caecum' 'Stomach'
 'Transverse colon' 'Rectum' 'Ascending colon' 'Descending colon'
 'Duodenum']

All segments combined - Round 1 Lemann:
ICC          0.821529
CI95%    [0.77, 0.87]
Name: 2, dtype: object


# Round 2

#### MaRIA round 2

In [4]:
# Segmental MARIA round 2

df = pd.read_csv("Inputs/round_2_maria_segmental.csv", header=0)

print(df['newBowelSegmentName'].unique())

# ILEUM AND JEJUNUM ARE A SINGLE SEGMENT HERE (terminal ileum is separate)
# However, there are some repetitions of segments, so we need to merge them and take the highest score
df['patient_scan_segment_scorer'] = df['patientIdentifier'] + ' ' + df['procedureName'] + ' ' + df['newBowelSegmentName'] + ' ' + df['scorerEmail']
df = df.sort_values('overallSegmentalScore', ascending=False)
df = df.drop_duplicates(subset=['patient_scan_segment_scorer'], keep='first')
print('\nSCORES count:', df.value_counts('patient_scan_segment_scorer').value_counts())
print('\nNEW bowel segment names:', df['newBowelSegmentName'].unique())

# make target to be patient + test + segment
df['patient_scan_segment'] = df['patientIdentifier'] + ' ' + df['procedureName'] + ' ' + df['newBowelSegmentName']
df = df[['patient_scan_segment', 'scorerEmail', 'overallSegmentalScore']]

all_segments_icc_r2_maria = pg.intraclass_corr(data=df, targets='patient_scan_segment', raters='scorerEmail',
                            ratings='overallSegmentalScore', nan_policy='omit')
all_segments_icc_r2_maria = all_segments_icc_r2_maria.loc[2,:]
print('\nRound 2 Maria:')
print(all_segments_icc_r2_maria[['ICC', 'CI95%']])

segments = ['Stomach', 'Duodenum', 'Jejunum', 'Ileum',  'Terminal ileum', 'Right colon', 'Transverse colon', 'Descending colon', 'Sigmoid colon', 'Rectum']

# initialize an empty DataFrame to store results
per_segment_r2_maria = pd.DataFrame(columns=['ICC', 'CI95%'], index=segments)

for segment in segments:
    # select only rows where patient_scan_segment contains the "segment" string
    df_segment = df[df['patient_scan_segment'].str.contains(segment)]
    if len(df_segment['overallSegmentalScore'].unique()) < 2:
        per_segment_r2_maria.at[segment, 'ICC'] = np.nan
        per_segment_r2_maria.at[segment, 'CI95%'] = f"[all scores = {df_segment['overallSegmentalScore'].unique()[0]}]"
        continue
    icc = pg.intraclass_corr(data=df_segment, targets='patient_scan_segment', raters='scorerEmail',
                            ratings='overallSegmentalScore', nan_policy='omit')
    icc = icc.loc[2,:]

    per_segment_r2_maria.at[segment, 'ICC'] = icc['ICC']
    per_segment_r2_maria.at[segment, 'CI95%'] = icc['CI95%']



['Descending colon' 'Duodenum' 'Ileum' 'Jejunum' 'Rectum' 'Right colon'
 'Sigmoid colon' 'Stomach' 'Terminal ileum' 'Transverse colon']

SCORES count: count
1    900
Name: count, dtype: int64

NEW bowel segment names: ['Terminal ileum' 'Ileum' 'Sigmoid colon' 'Jejunum' 'Rectum' 'Stomach'
 'Right colon' 'Transverse colon' 'Descending colon' 'Duodenum']

Round 2 Maria:
ICC          0.784087
CI95%    [0.74, 0.82]
Name: 2, dtype: object


#### Lemann round 2

In [5]:
# Segmental LEMANN round 2

df = pd.read_csv("Inputs/round_2_lemann_segmental.csv", header=0)

print(df['bowelSegmentName'].unique())

# ILEUM AND JEJUNUM ARE STRATIFIED INTO 10 SEGMENTS (terminal ileum = ILEUM 10)
# Change bowel segment names to be consistent with MARIA
df['bowelSegmentName'] = df['bowelSegmentName'].replace({'Ileum 10': 'Terminal ileum'})
df['bowelSegmentName'] = df['bowelSegmentName'].str.replace(r'\d+', '', regex=True)
df['bowelSegmentName'] = df['bowelSegmentName'].str.strip()

# Merge ileum and jejunum subsegments, picking the highest score
df['patient_segment_scorer'] = df['patientIdentifier'] + ' ' + df['procedureName'] + ' ' + df['bowelSegmentName'] + ' ' + df['scorerEmail']
df = df.sort_values('segmentIndex', ascending=False)
df = df.drop_duplicates(subset=['patient_segment_scorer'], keep='first')
print('\nSCORES count:', df.value_counts('patient_segment_scorer').value_counts())
print('\nNEW bowel segment names:', df['bowelSegmentName'].unique())

df['patient_scan_segment'] = df['patientIdentifier'] + ' ' + df['procedureName'] + ' ' + df['bowelSegmentName']
df = df[['patient_scan_segment', 'scorerEmail', 'segmentIndex']]
df['segmentIndex'] = df['segmentIndex'].astype(float)

all_segments_icc_r2_lemann = pg.intraclass_corr(data=df, targets='patient_scan_segment', raters='scorerEmail',
                            ratings='segmentIndex', nan_policy='omit')
all_segments_icc_r2_lemann = all_segments_icc_r2_lemann.loc[2,:]
print('\nRound 2 Lemann:')
print(all_segments_icc_r2_lemann[['ICC', 'CI95%']])

segments = ['Stomach', 'Duodenum', 'Jejunum', 'Ileum', 'Terminal ileum', 'Caecum', 'Ascending colon', 'Transverse colon', 'Descending colon', 'Sigmoid colon', 'Rectum']

# initialize an empty DataFrame to store results
per_segment_r2_lemann = pd.DataFrame(columns=['ICC', 'CI95%'], index=segments)

for segment in segments:
    # select only rows where patient_scan_segment contains the "segment" string
    df_segment = df[df['patient_scan_segment'].str.contains(segment)]
    if segment == 'Ileum':
        df_segment = df_segment[df_segment['segmentIndex'] != 'Ileum 10']
    if len(df_segment['segmentIndex'].unique()) < 2:
        per_segment_r2_lemann.at[segment, 'ICC'] = np.nan
        per_segment_r2_lemann.at[segment, 'CI95%'] = f"[all scores = {df_segment['segmentIndex'].unique()[0]}]"
        continue
    icc = pg.intraclass_corr(data=df_segment, targets='patient_scan_segment', raters='scorerEmail', ratings='segmentIndex', nan_policy='omit')
    icc = icc.loc[2,:]

    per_segment_r2_lemann.at[segment, 'ICC'] = icc['ICC']
    per_segment_r2_lemann.at[segment, 'CI95%'] = icc['CI95%']

# rename Ileum 10 to terminal ileum
per_segment_r2_lemann.rename(index={'Ileum 10': 'Terminal ileum'}, inplace=True)


['Ascending colon' 'Caecum' 'Descending colon' 'Duodenum' 'Ileum 1'
 'Ileum 10' 'Ileum 2' 'Ileum 3' 'Ileum 4' 'Ileum 5' 'Ileum 6' 'Ileum 7'
 'Ileum 8' 'Ileum 9' 'Jejunum 1' 'Jejunum 10' 'Jejunum 2' 'Jejunum 3'
 'Jejunum 4' 'Jejunum 5' 'Jejunum 6' 'Jejunum 7' 'Jejunum 8' 'Jejunum 9'
 'Rectum' 'Sigmoid colon' 'Stomach' 'Transverse colon']

SCORES count: count
1    990
Name: count, dtype: int64

NEW bowel segment names: ['Terminal ileum' 'Sigmoid colon' 'Ileum' 'Jejunum' 'Rectum'
 'Ascending colon' 'Transverse colon' 'Descending colon' 'Caecum'
 'Stomach' 'Duodenum']

Round 2 Lemann:
ICC          0.869089
CI95%    [0.84, 0.89]
Name: 2, dtype: object


# Validation

#### MaRIA validation

In [6]:
# Segmental MARIA validaiton
# 202 scan-scores, 10 segments - tehre should be 2020 scores

df = pd.read_csv("Inputs/validation_maria_segmental.csv", header=0)

print(df['newBowelSegmentName'].unique())

# ILEUM AND JEJUNUM ARE A SINGLE SEGMENT HERE (terminal ileum is separate)
# However, there are some repetitions of segments, so we need to merge them and take the highest score
df['patient_scan_segment_scorer'] = df['patientIdentifier'] + ' ' + df['procedureName'] + ' ' + df['newBowelSegmentName'] + ' ' + df['scorerEmail']
# print(df.value_counts('patient_scan_segment_scorer'))
df = df.sort_values('overallSegmentalScore', ascending=False)
df = df.drop_duplicates(subset=['patient_scan_segment_scorer'], keep='first')
# print(df.value_counts('patient_scan_segment_scorer'))
# print('\nSCORES count:', df.value_counts('patient_scan_segment_scorer').value_counts())
print('\nNEW bowel segment names:', df['newBowelSegmentName'].unique())

# make target to be patient + test + segment
df['patient_scan_segment'] = df['patientIdentifier'] + ' ' + df['procedureName'] + ' ' + df['newBowelSegmentName']
df = df[['patient_scan_segment', 'scorerEmail', 'overallSegmentalScore']]

df = df[df['patient_scan_segment'].map(df['patient_scan_segment'].value_counts()) > 1]
print('\nSCORES after dropping single-reported:', df.shape)

df.to_csv('Inputs/Intermediates/validation_maria_segmental_filtered.csv', index=False)


########### Analysis now in R ################


['Descending colon' 'Duodenum' 'Ileum' 'Jejunum' 'Rectum' 'Right colon'
 'Sigmoid colon' 'Stomach' 'Terminal ileum' 'Transverse colon']

NEW bowel segment names: ['Terminal ileum' 'Ileum' 'Transverse colon' 'Right colon' 'Sigmoid colon'
 'Descending colon' 'Duodenum' 'Stomach' 'Jejunum' 'Rectum']

SCORES after dropping single-reported: (1120, 3)


In [7]:
# Segmental LEMANN validation set

df = pd.read_csv("Inputs/validation_lemann_segmental.csv", header=0)

print(df['bowelSegmentName'].unique())

# ILEUM AND JEJUNUM ARE STRATIFIED INTO 10 SEGMENTS (terminal ileum = ILEUM 10)
# Change bowel segment names to be consistent with MARIA
df['bowelSegmentName'] = df['bowelSegmentName'].replace({'Ileum 10': 'Terminal ileum'})
df['bowelSegmentName'] = df['bowelSegmentName'].str.replace(r'\d+', '', regex=True)
df['bowelSegmentName'] = df['bowelSegmentName'].str.strip()

# Merge ileum and jejunum subsegments, picking the highest score
df['patient_segment_scorer'] = df['patientIdentifier'] + ' ' + df['procedureName'] + ' ' + df['bowelSegmentName'] + ' ' + df['scorerEmail']
# print(df.value_counts('patient_segment_scorer'))
df = df.sort_values('segmentIndex', ascending=False)
df = df.drop_duplicates(subset=['patient_segment_scorer'], keep='first')
# print('\nSCORES count:', df.value_counts('patient_segment_scorer').value_counts())
print('\nNEW bowel segment names:', df['bowelSegmentName'].unique())


df['patient_scan_segment'] = df['patientIdentifier'] + ' ' + df['procedureName'] + ' ' + df['bowelSegmentName']
df = df[['patient_scan_segment', 'scorerEmail', 'segmentIndex']]

df = df[df['patient_scan_segment'].map(df['patient_scan_segment'].value_counts()) > 1]
print('\nSCORES after dropping single-reported:', df.shape)

df['segmentIndex'] = df['segmentIndex'].astype(float)

df.to_csv('Inputs/Intermediates/validation_lemann_segmental_filtered.csv', index=False)

############### Analysis now in R ################


['Ascending colon' 'Caecum' 'Descending colon' 'Duodenum' 'Ileum 1'
 'Ileum 10' 'Ileum 2' 'Ileum 3' 'Ileum 4' 'Ileum 5' 'Ileum 6' 'Ileum 7'
 'Ileum 8' 'Ileum 9' 'Jejunum 1' 'Jejunum 10' 'Jejunum 2' 'Jejunum 3'
 'Jejunum 4' 'Jejunum 5' 'Jejunum 6' 'Jejunum 7' 'Jejunum 8' 'Jejunum 9'
 'Rectum' 'Sigmoid colon' 'Stomach' 'Transverse colon']

NEW bowel segment names: ['Terminal ileum' 'Transverse colon' 'Ileum' 'Ascending colon'
 'Descending colon' 'Sigmoid colon' 'Caecum' 'Jejunum' 'Duodenum'
 'Stomach' 'Rectum']

SCORES after dropping single-reported: (1232, 3)


In [8]:
# All segments validation MARIA

all_segments_icc_val_maria = pd.read_csv("Inputs/Intermediates/icc_all_segments_validation_maria.csv")
all_segments_icc_val_maria.rename(columns={'CI95': 'CI95%'}, inplace=True)


all_segments_icc_val_maria['CI95%'] = all_segments_icc_val_maria['CI95%'].str.replace(" ", ", ")
all_segments_icc_val_maria['CI95%'] = all_segments_icc_val_maria['CI95%'].apply(
    lambda x: [float(i) for i in x.strip("[]").split(",")]
)

all_segments_icc_val_maria = all_segments_icc_val_maria.transpose()
all_segments_icc_val_maria = all_segments_icc_val_maria.squeeze("columns")

all_segments_icc_val_maria


ICC             0.62916
CI95%    [0.594, 0.664]
Name: 0, dtype: object

In [9]:
# Per segment validation MARIA
per_segment_val_maria = pd.read_csv("Inputs/Intermediates/icc_per_segment_validation_maria.csv")
per_segment_val_maria.rename(columns={'CI95': 'CI95%'}, inplace=True)

# Function to convert CI string to list of floats, skip "all scores"
def parse_ci(x):
    if "all scores" in x:
        return x
    # Replace space with comma just in case, strip brackets
    x_clean = x.replace(" ", ",").strip("[]")
    return [float(i) for i in x_clean.split(",")]

# Apply
per_segment_val_maria['CI95%'] = per_segment_val_maria['CI95%'].apply(parse_ci)

per_segment_val_maria.set_index('Segment', inplace=True)


In [10]:
# All segments ICC validation Lemann

all_segments_icc_val_lemann = pd.read_csv("Inputs/Intermediates/icc_all_segments_validation_lemann.csv")
all_segments_icc_val_lemann.rename(columns={'CI95': 'CI95%'}, inplace=True)

all_segments_icc_val_lemann['CI95%'] = all_segments_icc_val_lemann['CI95%'].str.replace(" ", ", ")
all_segments_icc_val_lemann['CI95%'] = all_segments_icc_val_lemann['CI95%'].apply(
    lambda x: [float(i) for i in x.strip("[]").split(",")]
)

all_segments_icc_val_lemann = all_segments_icc_val_lemann.transpose()
all_segments_icc_val_lemann = all_segments_icc_val_lemann.squeeze("columns")

all_segments_icc_val_lemann


ICC            0.704208
CI95%    [0.675, 0.733]
Name: 0, dtype: object

In [11]:
# Per segment ICC validation Lemann
per_segment_val_lemann = pd.read_csv("Inputs/Intermediates/icc_per_segment_validation_lemann.csv")
per_segment_val_lemann.rename(columns={'CI95': 'CI95%'}, inplace=True)

# Function to convert CI string to list of floats, skip "all scores"
def parse_ci(x):
    if "all scores" in x:
        return x
    # Replace space with comma just in case, strip brackets
    x_clean = x.replace(" ", ",").strip("[]")
    return [float(i) for i in x_clean.split(",")]

# Apply function
per_segment_val_lemann['CI95%'] = per_segment_val_lemann['CI95%'].apply(parse_ci)

# Set index
per_segment_val_lemann.set_index('Segment', inplace=True)


# Visualise results

#### Segmental score overall

In [12]:
icc_results = pd.DataFrame(columns=['Round 1', 'Round 2'], index=['Segmental MaRIA','Segmental Lemann'])

icc_results.loc['Segmental MaRIA', 'Round 1'] = f'{all_segments_icc_r1_maria["ICC"]:.2f} [{all_segments_icc_r1_maria["CI95%"][0]} - {all_segments_icc_r1_maria["CI95%"][1]}]'
icc_results.loc['Segmental MaRIA', 'Round 2'] = f'{all_segments_icc_r2_maria["ICC"]:.2f} [{all_segments_icc_r2_maria["CI95%"][0]} - {all_segments_icc_r2_maria["CI95%"][1]}]'
icc_results.loc['Segmental Lemann', 'Round 1'] = f'{all_segments_icc_r1_lemann["ICC"]:.2f} [{all_segments_icc_r1_lemann["CI95%"][0]} - {all_segments_icc_r1_lemann["CI95%"][1]}]'
icc_results.loc['Segmental Lemann', 'Round 2'] = f'{all_segments_icc_r2_lemann["ICC"]:.2f} [{all_segments_icc_r2_lemann["CI95%"][0]} - {all_segments_icc_r2_lemann["CI95%"][1]}]'
icc_results.loc['Segmental MaRIA', 'Validation set'] = f'{all_segments_icc_val_maria["ICC"]:.2f} [{all_segments_icc_val_maria["CI95%"][0]} - {all_segments_icc_val_maria["CI95%"][1]}]'
icc_results.loc['Segmental Lemann', 'Validation set'] = f'{all_segments_icc_val_lemann["ICC"]:.2f} [{all_segments_icc_val_lemann["CI95%"][0]} - {all_segments_icc_val_lemann["CI95%"][1]}]'

icc_results

,Round 1,Round 2,Validation set
Segmental MaRIA,0.56 [0.46 - 0.66],0.78 [0.74 - 0.82],0.63 [0.594 - 0.664]
Segmental Lemann,0.82 [0.77 - 0.87],0.87 [0.84 - 0.89],0.70 [0.675 - 0.733]


In [13]:
import plotly.graph_objects as go
import plotly.subplots as sp
import plotly.express as px
import re
import numpy as np

# Define the ICC values and confidence intervals
icc_values = [
    all_segments_icc_r1_maria.loc['ICC'], all_segments_icc_r2_maria.loc['ICC'], all_segments_icc_val_maria.loc['ICC'],
    all_segments_icc_r1_lemann.loc['ICC'], all_segments_icc_r2_lemann.loc['ICC'], all_segments_icc_val_lemann.loc['ICC']
]  

conf_intervals = [
    all_segments_icc_r1_maria.loc['CI95%'],  all_segments_icc_r2_maria.loc['CI95%'], all_segments_icc_val_maria.loc['CI95%'],
    all_segments_icc_r1_lemann.loc['CI95%'],  all_segments_icc_r2_lemann.loc['CI95%'], all_segments_icc_val_lemann.loc['CI95%']
] 

# Calculate asymmetric errors
errors_lower = [icc_values[i] - conf_intervals[i][0] for i in range(len(icc_values))]
errors_upper = [conf_intervals[i][1] - icc_values[i] for i in range(len(icc_values))]

def color_to_rgba(color, alpha=1.0):
    if color.startswith('rgba'):
        return color
    if color.startswith('rgb'):
        nums = re.findall(r'\d+', color)
        return f'rgba({nums[0]},{nums[1]},{nums[2]},{alpha})'
    color = color.lstrip('#')
    return f'rgba({int(color[0:2],16)},{int(color[2:4],16)},{int(color[4:6],16)},{alpha})'

color1 = px.colors.qualitative.D3[0]
color2 = px.colors.qualitative.D3[2]

# Create a subplot with 1 row and 2 columns
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=("Segmental MaRIA", "Segmental Lemann"))
x_labels = ['Round 1', 'Round 2', 'Validation set']

# -- Segmental MaRIA --
fig.add_trace(go.Bar(
    x=x_labels,
    y=icc_values[:3],
    error_y=dict(
        type='data',
        symmetric=False,
        array=errors_upper[:3],
        arrayminus=errors_lower[:3],
        thickness=2,
        width=5
    ),
    marker=dict(
        color=color_to_rgba(color1, 0.55),
        line=dict(color=color1, width=2)
    ),
    name='Segmental MaRIA'
), row=1, col=1)

# -- Segmental Lemann --
fig.add_trace(go.Bar(
    x=x_labels,
    y=icc_values[3:],
    error_y=dict(
        type='data',
        symmetric=False,
        array=errors_upper[3:],
        arrayminus=errors_lower[3:],
        thickness=2,
        width=5
    ),
    marker=dict(
        color=color_to_rgba(color2, 0.55),
        line=dict(color=color2, width=2)
    ),
    name='Segmental Lemann'
), row=1, col=2)

# --- Annotations (ICC + CI values) ---
for col in [1, 2]:
    base_idx = 0 if col == 1 else 3
    for i, label in enumerate(x_labels):
        idx = base_idx + i
        icc = icc_values[idx]
        ci_lower, ci_upper = conf_intervals[idx]

        # ICC value on bar
        fig.add_annotation(
            x=label, y=icc,
            text=f"{icc:.2f}",
            showarrow=False,
            yshift=10,
            font=dict(size=11),
            row=1, col=col
        )

        # Upper bound label
        fig.add_annotation(
            x=label, y=ci_upper,
            text=f"{ci_upper:.2f}",
            showarrow=False,
            yshift=8,
            font=dict(size=11),
            row=1, col=col
        )

        # Lower bound label
        fig.add_annotation(
            x=label, y=ci_lower,
            text=f"{ci_lower:.2f}",
            showarrow=False,
            yshift=-10,
            font=dict(size=11),
            row=1, col=col
        )

# Horizontal line at y = 0.5
for i in range(1, 3):
    fig.add_shape(
        type="line",
        x0=-0.5, x1=2.5,
        y0=0.5, y1=0.5,
        xref=f'x{i}',
        yref=f'y{i}',
        line=dict(color="gray", width=2, dash="dot"),
    )

# Layout and axis settings
fig.update_layout(
    template="simple_white",
    yaxis_title="ICC",
    width=900,
    height=500,
    showlegend=False
)

fig.update_yaxes(range=[0, 1], showgrid=True)
fig.update_xaxes(range=[-0.5, 2.5])
fig.update_yaxes(tickvals=np.arange(0, 1.1, 0.1), ticktext=[f"{i:.1f}" for i in np.arange(0, 1.1, 0.1)])

fig.show()


#### Terminal ileum

In [14]:
# combine all the per_segment dataframes into one

per_segment_r1_maria['Scoring Round'] = 'MaRIA Round 1'
per_segment_r1_lemann['Scoring Round'] = 'Lemann Round 1'
per_segment_r2_maria['Scoring Round'] = 'MaRIA Round 2'
per_segment_r2_lemann['Scoring Round'] = 'Lemann Round 2'
per_segment_r1_maria.reset_index(inplace=True)
per_segment_r1_lemann.reset_index(inplace=True)
per_segment_r2_maria.reset_index(inplace=True)
per_segment_r2_lemann.reset_index(inplace=True)
per_segment_r1_maria.rename(columns={'index': 'Segment'}, inplace=True)
per_segment_r1_lemann.rename(columns={'index': 'Segment'}, inplace=True)
per_segment_r2_maria.rename(columns={'index': 'Segment'}, inplace=True)
per_segment_r2_lemann.rename(columns={'index': 'Segment'}, inplace=True)

# Add scoring round labels for validation
per_segment_val_maria['Scoring Round'] = 'MaRIA Validation'
per_segment_val_lemann['Scoring Round'] = 'Lemann Validation'

# Reset index and rename for validation as well
per_segment_val_maria.reset_index(inplace=True)
per_segment_val_lemann.reset_index(inplace=True)
per_segment_val_maria.rename(columns={'index': 'Segment'}, inplace=True)
per_segment_val_lemann.rename(columns={'index': 'Segment'}, inplace=True)

# Combine all dataframes, including validation
per_segment_combined = pd.concat([
    per_segment_r1_maria,
    per_segment_r1_lemann,
    per_segment_r2_maria,
    per_segment_r2_lemann,
    per_segment_val_maria,
    per_segment_val_lemann
], ignore_index=True)

# Reorder the columns
per_segment_combined = per_segment_combined[['Scoring Round', 'Segment', 'ICC', 'CI95%']]


In [15]:
df_temp = per_segment_combined.copy()

df = df_temp[df_temp['Segment'] == 'Terminal ileum']

df.reset_index(drop=True, inplace=True)
#df['lower_limit'] = df['CI95%']

for i in df.index:
    lower_limit = df.loc[i, 'CI95%'][0]
    upper_limit = df.loc[i, 'CI95%'][1]
    df.loc[i, 'lower_limit'] = lower_limit
    df.loc[i, 'upper_limit'] = upper_limit

df['error_lower'] = df['ICC'] - df['lower_limit']
df['error_upper'] = df['upper_limit'] - df['ICC']

df['system'] = np.where(df['Scoring Round'].str.contains('MaRIA'), 'MaRIA', 'Lemann')
df['round'] = np.where(
    df['Scoring Round'].str.contains('Validation'), 'Validation', 
    np.where(df['Scoring Round'].str.contains('Round 1'), 'Round 1', 'Round 2')
)

df.drop(columns=['Segment', 'CI95%'], inplace=True)

df.set_index('Scoring Round', inplace=True, drop=True)

# Ensure desired order
df['system'] = pd.Categorical(df['system'], categories=['MaRIA', 'Lemann'], ordered=True)
df['round'] = pd.Categorical(df['round'], categories=['Round 1', 'Round 2', 'Validation'], ordered=True)
df = df.sort_values(by=['system', 'round']).copy()

print(df)

def color_to_rgba(hex_color, alpha):
    """Convert hex color to rgba with transparency."""
    hex_color = hex_color.lstrip('#')
    rgb = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
    return f'rgba({rgb[0]}, {rgb[1]}, {rgb[2]}, {alpha})'

# Define custom colors
color1 = '#1f77b4'  # Blue for MaRIA
color2 = '#2ca02c'  # Green for Lemann

# Prepare data
x_labels = ['Round 1', 'Round 2', 'Validation set']
icc_values = df['ICC'].tolist()
errors_lower = df['error_lower'].tolist()
errors_upper = df['error_upper'].tolist()
conf_intervals = list(zip(df['lower_limit'], df['upper_limit']))

# Create subplot with titles
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=("Terminal ileum - MaRIAs", "Terminal ileum - Lemann"))

# MaRIA (first 3)
fig.add_trace(go.Bar(
    x=x_labels,
    y=icc_values[:3],
    error_y=dict(
        type='data',
        symmetric=False,
        array=errors_upper[:3],
        arrayminus=errors_lower[:3],
        thickness=2,
        width=5
    ),
    marker=dict(
        color=color_to_rgba(color1, 0.55),
        line=dict(color=color1, width=2)
    ),
    name='Segmental MaRIA'
), row=1, col=1)

# Lemann (last 3)
fig.add_trace(go.Bar(
    x=x_labels,
    y=icc_values[3:],
    error_y=dict(
        type='data',
        symmetric=False,
        array=errors_upper[3:],
        arrayminus=errors_lower[3:],
        thickness=2,
        width=5
    ),
    marker=dict(
        color=color_to_rgba(color2, 0.55),
        line=dict(color=color2, width=2)
    ),
    name='Segmental Lemann'
), row=1, col=2)

# Add annotations (ICC, CI values)
for col in [1, 2]:
    base_idx = 0 if col == 1 else 3
    for i, label in enumerate(x_labels):
        idx = base_idx + i
        icc = icc_values[idx]
        ci_lower, ci_upper = conf_intervals[idx]

        # ICC value on bar
        fig.add_annotation(
            x=label, y=icc,
            text=f"{icc:.2f}",
            showarrow=False,
            yshift=10,
            font=dict(size=11),
            row=1, col=col
        )

        # Upper bound label
        fig.add_annotation(
            x=label, y=ci_upper,
            text=f"{ci_upper:.2f}",
            showarrow=False,
            yshift=8,
            font=dict(size=11),
            row=1, col=col
        )

        # Lower bound label
        fig.add_annotation(
            x=label, y=ci_lower,
            text=f"{ci_lower:.2f}",
            showarrow=False,
            yshift=-10,
            font=dict(size=11),
            row=1, col=col
        )

# Add horizontal reference line at ICC = 0.5
for i in range(1, 3):
    fig.add_shape(
        type="line",
        x0=-0.5, x1=2.5,
        y0=0.5, y1=0.5,
        xref=f'x{i}',
        yref=f'y{i}',
        line=dict(color="gray", width=2, dash="dot"),
    )

# Final layout tweaks
fig.update_layout(
    template="simple_white",
    yaxis_title="ICC",
    width=900,
    height=500,
    showlegend=False
)

fig.update_yaxes(range=[0, 1], showgrid=True)
fig.update_xaxes(range=[-0.5, 2.5])
fig.update_yaxes(tickvals=np.arange(0, 1.1, 0.1),
                 ticktext=[f"{i:.1f}" for i in np.arange(0, 1.1, 0.1)])

fig.show()


                        ICC  lower_limit  upper_limit error_lower error_upper  \
Scoring Round                                                                   
MaRIA Round 1      0.479516        0.170        0.820    0.309516    0.340484   
MaRIA Round 2      0.694214        0.510        0.850    0.184214    0.155786   
MaRIA Validation   0.676236        0.570        0.774    0.106236    0.097764   
Lemann Round 1     0.835177        0.630        0.960    0.205177    0.124823   
Lemann Round 2     0.873025        0.770        0.940    0.103025    0.066975   
Lemann Validation  0.647426        0.537        0.752    0.110426    0.104574   

                   system       round  
Scoring Round                          
MaRIA Round 1       MaRIA     Round 1  
MaRIA Round 2       MaRIA     Round 2  
MaRIA Validation    MaRIA  Validation  
Lemann Round 1     Lemann     Round 1  
Lemann Round 2     Lemann     Round 2  
Lemann Validation  Lemann  Validation  
